# Function Testing Notebook - Joseph

This notebook is meant to test external-facing functions to ensure they are working as expected.

A dedicated test_all_functions.py with unit testing might be better, but this is a good benchmark

In [1]:
import logging
import os
import sys
from pathlib import Path
import json

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from okabeito import black, blue, green, lightblue, orange, purple, red, yellow
from seaborn import axes_style

from pythoneeg import constants, core, visualization


/mnt/isilon/marsh_single_unit/PythonEEG/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tempdir = Path('../../tmp').resolve()
core.set_temp_directory(tempdir)
print(tempdir)

/mnt/isilon/marsh_single_unit/PythonEEG/tmp


In [3]:
# Set up logger with detailed formatting
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO, stream=sys.stdout, force=True)
logger = logging.getLogger()


In [4]:
logger.setLevel(logging.INFO)
wars = []

animal_ids = [p for p in Path("./test-wars-sox5-7").glob("*") if p.is_dir()]
# bad_animal_ids = [
#     "013122_cohort4_group7_2mice both_FHET FHET(2)",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT MHET",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT MMUT",
#     "011622_cohort4_group4_3mice_MMutOLD_FMUT_FMUT_FWT OLDMMT",
#     "011322_cohort4_group3_4mice_AllM_MT_WT_HET_WT M3",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT FHET"
#     # "060921_Cohort 3_EM1_AM2_GF4 AM2" # this seems fine just very high delta for whatever reason
# ]

# Create empty lists to store data
df = []

for i, animal_id in enumerate(animal_ids):
    json_files = list(Path(animal_id).glob('*.json'))
    if len(json_files) > 1:
        print(f"Multiple json files found for {animal_id}")
        break
    json_file = json_files[0]

    # Load the JSON file
    with open(json_file, 'r') as f:
        jsonfile = json.load(f)
        data = jsonfile['bad_channels_dict']
        for day in data.keys():
            df.append({"animal" : animal_id.name, 'genotype' : jsonfile['genotype'], 'animalday' : day, 'n_bad' : len(data[day]), 'keep?' : None, "comments" : None})

# Create dataframe
df = pd.DataFrame(df)
df = df.sort_values(['animal', 'animalday'])
display(df)


,animal,genotype,animalday,n_bad,keep?,comments
50,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-...",FHet,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-R...",0,None,None
52,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-...",FHet,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-R...",0,None,None
51,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-...",FHet,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-R...",0,None,None
49,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-...",FHet,"032221_cohort 2, Group 3, Mouse 6 Cage 2A Re-R...",0,None,None
105,040221_Group 1 Mouse 9 Recording,FMut,040221_Group 1 Mouse 9 Recording FMut Apr-02-2021,0,None,None
...,...,...,...,...,...,...
97,121821_cohort4_Group1_2mice M2,Unknown,M2 Unknown Dec-21-2021,0,None,None
85,121821_cohort4_Group1_2mice M4,Unknown,M4 Unknown Dec-18-2021,0,None,None
84,121821_cohort4_Group1_2mice M4,Unknown,M4 Unknown Dec-19-2021,0,None,None
83,121821_cohort4_Group1_2mice M4,Unknown,M4 Unknown Dec-20-2021,2,None,None


In [5]:
len(df['animal'].unique())

70

In [6]:
save_folder = '/home/dongjp/Downloads/8-4 fine filtering'
save_files = True
save_folder = Path(save_folder)
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
display(save_folder)

PosixPath('/home/dongjp/Downloads/8-4 fine filtering')

In [7]:
df.to_excel(save_folder / f'{save_folder.name}.xlsx', index=False)